In [51]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from PIL import Image
import random
import math
import matplotlib.pyplot as plt

In [37]:
training_data = np.load('./output/training_data.npy')
training_labels = np.load('./output/training_dists.npy')

#pad the training data matrix so it is 100x1024

#(500, 100, 1000, 1)

zero_padding_array = np.zeros((500,100,600,1))

In [38]:
training_data.shape

(500, 100, 1000, 1)

In [39]:
training_labels.shape

(500, 100, 100, 1)

In [40]:
zero_padding_array.shape

(500, 100, 600, 1)

In [41]:
print(training_data.shape)
training_data = np.concatenate([training_data, zero_padding_array], axis=2)
print(training_data.shape)

print(training_labels.shape)

samples_count = training_data.shape[0]

train_size = math.floor(0.85*training_data.shape[0])

shuffle_indices = random.sample(range(0, samples_count), samples_count)

indices_train = shuffle_indices[0:train_size]
indices_test = shuffle_indices[train_size+1:samples_count]

X_train = training_data[indices_train,:]
Y_train = training_labels[indices_train]

X_test = training_data[indices_test,:]
Y_test = training_labels[indices_test]

print('X_train.shape : ', X_train.shape)
print('X_test.shape : ', X_test.shape)

print('Y_train.shape : ', Y_train.shape)
print('Y_test.shape : ', Y_test.shape)

np.save('X_test.npy', X_test)
np.save('Y_test.npy', Y_test)

model = Sequential()

# Layer 1
model.add(Conv2D(64, input_shape=(100,1600,1), kernel_size=(2,1), strides=(1,1), padding='same'))
#model.add(Conv2D(48, input_shape=(400,11,1), kernel_size=(2,2), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))

# Layer 2
model.add(Conv2D(32, (2, 1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))

# Layer 3
model.add(Conv2D(16, (2, 1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))

# Layer 4
model.add(Conv2D(1, (2, 1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))

model.summary()

(500, 100, 1000, 1)
(500, 100, 1600, 1)
(500, 100, 100, 1)
X_train.shape :  (425, 100, 1600, 1)
X_test.shape :  (74, 100, 1600, 1)
Y_train.shape :  (425, 100, 100, 1)
Y_test.shape :  (74, 100, 100, 1)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 100, 1600, 64)     192       
_________________________________________________________________
batch_normalization_9 (Batch (None, 100, 1600, 64)     256       
_________________________________________________________________
activation_9 (Activation)    (None, 100, 1600, 64)     0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 100, 800, 64)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 100, 800, 32)      4128      
___________________________________________________

In [42]:
# (4) Compile 
#model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])

In [43]:
# (5) Train
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=16, epochs=15, verbose=1)

model.save('./output/CNN_model_03.h5')

Train on 425 samples, validate on 74 samples
Epoch 1/15
425/425 [==============================] - 103s 241ms/step - loss: 0.2664 - accuracy: 0.6448 - val_loss: 0.2234 - val_accuracy: 0.6608
Epoch 2/15
425/425 [==============================] - 102s 240ms/step - loss: 0.2358 - accuracy: 0.6446 - val_loss: 0.2236 - val_accuracy: 0.6608
Epoch 3/15
425/425 [==============================] - 102s 240ms/step - loss: 0.2198 - accuracy: 0.6447 - val_loss: 0.2236 - val_accuracy: 0.6608
Epoch 4/15
425/425 [==============================] - 102s 240ms/step - loss: 0.2151 - accuracy: 0.6446 - val_loss: 0.2235 - val_accuracy: 0.6608
Epoch 5/15
425/425 [==============================] - 101s 239ms/step - loss: 0.2128 - accuracy: 0.6448 - val_loss: 0.2234 - val_accuracy: 0.6608
Epoch 6/15
425/425 [==============================] - 101s 238ms/step - loss: 0.2120 - accuracy: 0.6448 - val_loss: 0.2234 - val_accuracy: 0.6608
Epoch 7/15
425/425 [==============================] - 101s 239ms/step - loss: 0

In [46]:
training_result = model.evaluate(X_train, Y_train)
testing_result = model.evaluate(X_test, Y_test)

74/74 [==============================] - 5s 71ms/step


In [47]:
print('Training Acc: ', training_result[1])
print('Testing Acc: ', testing_result[1])

Training Acc:  0.645237147808075
Testing Acc:  0.6607540249824524


In [48]:
results = model.predict(X_test)

In [ ]:
plt.plot(history.history['loss'])
plt.show()